### Challenge Stokes

##### Import Libraries

In [1]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot, cm
from math import pi
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

#### Parametres

In [2]:
xmin = 0.
ymin = 0.

l = 1.
h = 1.
u = 1.

nx = 41
ny = 41

# Return evenly spaced numbers over a specified interval
x=numpy.linspace(xmin,l,nx)
y=numpy.linspace(ymin,h,ny)
X,Y = numpy.meshgrid(x,y)    # Grille


# Spacing

dx = l/(nx-1)
dy = h/(ny-1)

l1_target = 1e-6

# Initial conditions (Initial guess)
p = numpy.zeros((ny,nx)) #create a XxY vector of 0's
w = numpy.zeros((ny,nx))

In [3]:
def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

##### Iterations Poisson

In [4]:
def poisson_2d(p, w, dx, dy, l1_target):
    '''Performs Jacobi relaxation
    
    Parameters:
    ----------
    p : 2D array of floats
        Initial guess
    b : 2D array of floats
        Source term
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction
    l1_target: float
        Target difference between two consecutive iterates
    
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    '''

    norm1=1   #Normes initiales pour lancer la boucle
    norm2=1
    iterations = 0
   
    
    #numpy.empty_like(a, dtype=None, order='K', subok=True)
    #Return a new array with the same shape and type as a given array.
    
    pd = numpy.empty_like(p)
    wn = numpy.empty_like(w)
    
    # Initial guess for p
    wn = numpy.empty_like(p)
   
    while  norm1> l1_target or norm2>l1_target :

        pd = p.copy()                 # On place la valeur de p dans pd
        wn = w.copy()                 # On place la valeur de w dans wn
      
        # Solutions itératives 
            
        # Pour w
        # On ne commence pas au bord de la grille mais juste après
        w[1:-1,1:-1] = 0.25*(wn[:-2,1:-1]+wn[2:,1:-1]+wn[1:-1,:-2]+wn[1:-1,2:])
            
        # Pour p
            
        p[1:-1,1:-1] = 0.25*(pd[1:-1,2:]+pd[1:-1,0:-2]+pd[2:,1:-1]+pd[0:-2,1:-1]+w[1:-1,1:-1]*(dx)**2)
        
        # "Garde fou" si erreur dans le code
        if iterations > 50000 :
            break    
                
                # Boundary conditions for w

            # Au-dessus
        w[-1,:]=(-1/(2*(dx)**2))*(8*p[-2,:]-p[-3,:])-3*u/dx

            # En-dessous
        w[0,:]=(-1/(2*(dx)**2))*(8*p[1,:]-p[2,:])  #les termes précédents ici sont j+1 et j+2, non plus j-1 et j-2

            # A gauche
        w[:,0]=(-1/(2*(dx)**2))*(8*p[:,1]-p[:,2])  #les termes précédents ici sont i+1 et i+2

            # A droite
        w[:,-1]=(-1/(2*(dx)**2))*(8*p[:,-2]-p[:,-3])  

    
        
        # BCs are automatically enforced
        
        norm1=L1norm(pd,p)
        norm2=L1norm(wn,w)
        iterations += 1
    
    print('Number of Jacobi iterations: {0:d}'.format(iterations))
    return p   

In [5]:
p = poisson_2d(p, w, dx, dy, l1_target)

print(p)

Number of Jacobi iterations: 50001
[[  0.   0.   0. ...,   0.   0.   0.]
 [  0. -inf  inf ...,  inf -inf   0.]
 [  0.  inf -inf ..., -inf  inf   0.]
 ..., 
 [  0.  inf -inf ..., -inf  inf   0.]
 [  0. -inf  inf ...,  inf -inf   0.]
 [  0.   0.   0. ...,   0.   0.   0.]]


/serveur/logiciels/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:46: RuntimeWarning: overflow encountered in add
